# Implementação do Algoritmo Simplex 

In [1]:
# Bibliotecas
import math
import numpy as np
import time

In [2]:
# Funcao para imprimir tableu
def print_tableau(tableau):
    lins = len(tableau)
    cols = len(tableau[0])
    
    header =  ['x' + str(i+1) for i in range(cols-1)]
    print('           ' + ''.join(f'{col:>8}' for col in header))
    print('              ' + '-' * (7 * cols))
    
    print('F.O. ' + ''.join(f'{val:8.2f}' for val in tableau[0]))
    for i in range(1,lins-1):
        print(f'x{tableau[-1][i-1]+1} = ' + ''.join(f'{val:8.2f}' for val in tableau[i]))
    
    print()

# Fase 1: obter solução básica viável

In [3]:
# Procura alguma coluna da matriz Identidade
def eh_basica(coluna):
    return sum(coluna) == 1 and len([c for c in coluna if c == 0]) == len(coluna) - 1

# Retorna o indice onde esta "1"
def get_one(vector):
    for i in range(len(vector)):
        if vector[i] == 1:
            return i

# Valores muito proximos de zero
def is_zero(element):
    return abs(element) <= 10e-4

# Funcao que guarda o indice das variaveis basicas
# associado a coluna da matriz identidade
def basicas(A):
    id = [0] * len(A)       # coluna da identidade
    i_B = [-1] * len(A)     # indice da variavel
    colunas = np.array(A).T
    for coluna in range(0, len(colunas)):
        if eh_basica(colunas[coluna]) and id[get_one(colunas[coluna])] == 0:
            i_B[get_one(colunas[coluna])]= coluna
            id[get_one(colunas[coluna])] = 1
    return id, i_B
# OBS: quando escreve basicas(A)[0] --> acessa quais colunas
#                     basicas(A)[1] --> acessa os indices (-1) das basicas

def fase1(c, A, b):
    nvar = len(A[0])
    d = (False, nvar)
    # Verifica se é necessario adicionar variaveis artificiais
    tem_sol_basica = any(x == -1 for x in basicas(A)[1])
    if not tem_sol_basica:
        d = (False, nvar)
        return c, A, b, d
 
    # Custos da nova funcao
    c1 = [0] * len(c)    # zero para as variaveis originais
    id = basicas(A)[0]
    for i in range(0, len(id)):
        if id[i] == 0:
            coluna = [0] * len(id)
            coluna[i] = 1
            for linha in range(0, len(A)):
                A[linha].append(coluna[linha])
            c1.append(1)  # um para as variaveis artificiais
    
    # se precisou adicionar variaveis
    d = (True, nvar)
    
    return c1, A, b, d

A fase 1 deve retornar um Tableau !!

Usamos somente as colunas:

    - da funcao objetivo e valores das basicas
    - das variaveis originais
    
OBS: se a solucao encontrada for degenerada entao excluimos a linha de zeros

In [4]:
# Reorganiza o tableau
def resultado_fase1(tableau, nvar):
    # mantem as colunas das variaveis originais
    if not is_zero(tableau[0][0]):
        return None, None

    qtdd_colunas = nvar + 1

    # Verify tableau has valid dimensions before accessing
    if len(tableau) < 2:
        return None, None

    # Obtem os indices das variaveis basicas
    basic_vars = tableau[-1] if len(tableau[-1]) > 0 else []

    novo_A = []
    novo_b = []

    # Processa cada linha, exceto a primeira e a última
    for basica in range(1, len(tableau)-1):
        if basica < len(tableau) and basica < len(basic_vars) and basic_vars[basica] > nvar-1:
            for j in range(1, nvar+1):
                if j < len(tableau[basica]) and not is_zero(tableau[basica][j]):
                    tableau = atualiza_tableau(tableau, (basica, j))

    # mantem as linhas das variaveis basicas originais
    if len(basic_vars) > 0:
        for i, xB in enumerate(basic_vars):
            if i+1 < len(tableau) and xB < qtdd_colunas:
                row = tableau[i+1]
                if len(row) >= qtdd_colunas:
                    novo_A.append(row[1:qtdd_colunas])
                    novo_b.append(row[0])

    return novo_A, novo_b

# Fase 2: Aplicar Método Simplex

## Tableau Inicial 

Estamos assumindo que para montar o Tableau está sendo passada uma matriz A que contenha todas as colunas da identidade necessarias para obter uma solucao basica viável.

OBS: garantimos isso pois antes todos os problemas passam pela fase 1


In [5]:
# Monta o Tableau Inicial se passou pela Fase 1
def tableau_inicial_simples(c, A, b):
    linhas_tableau = []
    
    # guarda as basicas
    identidade, indice_basicas = basicas(A)
    
    # funcao objetivo e custos reduzidos
    funcao_obj = (-1) * sum(b)
    custos = []
    for j in range(len(A[0])):
        if j not in indice_basicas:
            c_j = 0
            for i in range(len(A)):
                c_j += A[i][j]
            custos.append((-1)*c_j)
        else: 
            custos.append(0)
    
    linhas_tableau.append([funcao_obj]+custos)  # linha 0

    # Concatena linhas de A com linhas de b
    for i in range(len(b)):
        linha = [b[i]] + A[i]
        linhas_tableau.append(linha)

    # indice das variaveis basicas
    linhas_tableau.append(indice_basicas)  # ultima linha
    return linhas_tableau

In [6]:
# Monta o Tableau Inicial se nao passou pela Fase 1
def tableau_inicial(c, A, b):
    linhas_tableau = []
    
    # guarda as basicas
    identidade, indice_basicas = basicas(A)
    lz = []
    
     # funcao objetivo e custos reduzidos
    fo = 0
    for i in range(0, len(b)):
        fo += -(b[i] * c[indice_basicas[i]])   
    lz.append(fo)

    # Percorre todas as colunas da matriz
    for i in range(0, len(A[0])):
        # se a variavel nao é basica, calcula o custo reduzido
        if i not in indice_basicas:
            col = [A[j][i] for j in range(0, len(A))] 
            cust = [c[i] for i in indice_basicas]
            el = sum([a*b for a,b in zip(col,cust)])
            el = c[i] - el
            lz.append(el)
        # se é basica, entao custo é zero
        else:
            lz.append(0)

    linhas_tableau.append(lz)  # linha 0

    # Concatena linhas de A com linhas de b
    for i in range(len(b)):
        linha = [b[i]] + A[i]
        linhas_tableau.append(linha)

    # indice das variaveis basicas
    linhas_tableau.append(indice_basicas)  # ultima linha
    return linhas_tableau

Funções que verificam se a solução é ótima ou não e atualizam o Tableau, se necessário

In [7]:
# Verifica se tem algum custo reduzido negativo
def nao_otima(tableau):
    return any(c < 0 for c in tableau[0][1:])

# Define elemento pivo
def posicao_pivo(tableau):
    custos = tableau[0]
    # Escolhe o primeiro custo reduzido negativo
    # "xj entra na base"
    for j in range(1,len(custos)):
        if custos[j] < 0:
            coluna_pivo = j
            break

    theta = []
    for linha_i in range (1,len(tableau)-1):
        if tableau[linha_i][coluna_pivo] <= 0:
            theta.append(math.inf)
        else:
            theta.append(tableau[linha_i][0] / tableau[linha_i][coluna_pivo])
        
    # Escolhe o menor indice das variaveis que podem sair da base
    # soma um porque estamos ignorando a linha de custos
    #print("Theta: ",theta)

    if set(theta) == {math.inf}:
        print("Problema ilimitado")
        return None, None

    linha_pivo = theta.index(min(theta)) + 1
    
    return linha_pivo, coluna_pivo

# Atualiza o tableau com base no pivo encontrado
def atualiza_tableau(tableau, pivo):
    i, j = pivo
    valor_pivo = tableau[i][j]
    
    # Divide a linha do pivo para que o valor do pivo seja 1
    tableau[i] = np.array(tableau[i]) / valor_pivo
    
    # Atualiza os valores de dentro do tableau
    # ou seja, zera os outros elementos da coluna
    for linha in range (len(tableau)-1):
        if linha != i:
            multiplicador = (-1) * tableau[linha][j]
            tableau[linha] = multiplicador*tableau[i] + tableau[linha]

            tableau[linha] = np.array([0 if is_zero(x) else x for x in list(tableau[linha])])
   
    # Atualiza variaveis básicas
    tableau[-1][i-1] = j-1 

    return tableau

def otima(tableau, nvar):
    # inicia com todos valendo zero
    solucoes = [0] * nvar
    
    # coloca o valor das basicas na posicao correta da solucao
    for i, xB in enumerate(tableau[-1]):
        solucoes[xB] = tableau[i+1][0]
        
    return solucoes        



In [8]:
"""
def simplex(c, A, b, d, t_fase1=False):
    if t_fase1:
        tableau = tableau_inicial(c, A, b)
    else:
        tableau = tableau_inicial_simples(c, A, b)

    nvar = d[1]
    
    it = 0
    while nao_otima(tableau):
        pivo = posicao_pivo(tableau)
        tableau = atualiza_tableau(tableau, pivo)
        it += 1
    print("Iterações:", it)

    if d[0]:
        novo_A, novo_b = resultado_fase1(tableau, nvar)
        # Check infeasibility based on Phase 1 objective value
        if infactivel(tableau):
            print("Problema infativel")
            return None, None
        return novo_A, novo_b
    else:
        # Return the optimal solution
        return A, otima(tableau, nvar)# SIMPLEX
"""


def simplex(c, A, b, d, t_fase1 = False):
    if t_fase1:
        tableau = tableau_inicial(c, A, b)
    else:
        tableau = tableau_inicial_simples(c, A, b)

    #print("Tableau inicial: ")
    #print_tableau(tableau)

    nvar = d[1]
    
    it = 0
    while nao_otima(tableau) and it < 10000:
        #print_tableau(tableau)
        it = it +1
        pivo = posicao_pivo(tableau)
        if pivo == (None, None):
            return None, None
        tableau = atualiza_tableau(tableau, pivo)
    print("Iterações: ",it)
        #print("Tableau atualizado: ")
        #print_tableau(tableau)

    if it == 10000:
        print("Max Iteracoes atingida")
        return None, None



    if d[0]:
        novo_A, novo_b = resultado_fase1(tableau, nvar)
        return novo_A, novo_b
    else:
        #print("Tableau final")
        #print_tableau(tableau)
        return A, otima(tableau, nvar)  


## Retorna a solução ótima

In [9]:
def get_solution_simplex(c, A, b):
    inicio = time.time()
    nvar = len(A[0])
    d = (False, nvar)
    c1, A1, b1, d = fase1(c, A, b)

    # Caso seja necessario usar variaveis artificiais (Fase 1)
    if d[0]:
        #print("Simplex FASE 1: ")
        A2, b2 = simplex(c1,A1,b1,d)
        if A2 is None:
            print("Problema infactivel")
            return None
        #else:
            #print("Resultado da fase 1: ",b2)

        # Convertendo tudo para mesmo formato que o simplex (Fase 2)
        A2 = [[float(x) for x in row] for row in A2]
        b2 = [float(x) for x in b2] 
        
        fA, fb = simplex(c, A2, b2,(False, nvar), True)
        print("Solução ótima: ",fb)
        
    # Não foi necessario usar a fase 1 (Resolve direto Fase 2)    
    else:  
        fA, fb = simplex(c, A1, b1, d)
        print("Solução ótima: ", fb)
    
    fim = time.time()
    print("Tempo de execução: ", fim - inicio)

# Testando alguns exemplos #

### Slides aula 23

Solucão básica viável para iniciar o simplex:
(x1,x2,x3,x4) = (1 ; 1/2 ; 1/3 ; 0) e zero nas variaveis artificiais

Problema: A tem linhas LD --> sol. encontrada na fase 1 é degenerada

In [10]:
# Coeficientes da função objetivo
c = [1, 1, 1, 0]

# Matriz de coeficientes das restrições
A = [
    [1, 2, 3, 0],
    [-1, 2, 6, 0],
    [0, 4, 9, 0],
    [0, 0, 3, 1]
]

# Vetor de termos independentes das restrições
b = [3, 2, 5, 1]

get_solution_simplex(c, A, b)

Iterações:  3
Iterações:  1
Solução ótima:  [np.float64(0.4999999999999998), np.float64(1.2500000000000002), 0, np.float64(1.0000000000000004)]
Tempo de execução:  0.0018198490142822266


### Sei la
Solução ótima: (3,2,0,0)

In [11]:
# Coeficientes da função objetivo
c = [-2, -1, 0, 0] 

# Matriz de coeficientes das restrições
A = [
    [-1, 1, 1, 0],  
    [1, 2, 0, 1]   
]

# Vetor de termos independentes das restrições
b = [-1, 7]

get_solution_simplex(c, A, b)

Iterações:  2
Solução ótima:  [np.float64(3.0), np.float64(2.0), 0, 0]
Tempo de execução:  0.0009310245513916016


### Lista 20
Solução ótima: (1,5,0)

In [12]:
# Coeficientes da função objetivo
c = [4, 3, 7] 

# Matriz de coeficientes das restrições
A = [
    [2, 2, 1], 
    [3, 1, 2]  
]

# Vetor de termos independentes das restrições
b = [12, 8]

get_solution_simplex(c, A, b)

Iterações:  2
Iterações:  0
Solução ótima:  [1.0, 5.0, 0]
Tempo de execução:  0.00045013427734375


### Lista 20
Trocando b para (5,8)

Solução ótima: (2,0,1)

In [13]:
# Coeficientes da função objetivo
c = [4, 3, 7]

# Matriz de coeficientes das restrições
A = [
    [2, 2, 1], 
    [3, 1, 2]  
]

# Vetor de termos independentes das restrições
b = [5, 8]

get_solution_simplex(c, A, b)

Iterações:  2
Iterações:  0
Solução ótima:  [2.0, 0, 1.0]
Tempo de execução:  0.001558065414428711


### Lista 16
Solução: problema infactível

In [14]:
# Coeficientes da função objetivo
c = [2, -1, 5, 0] 

# Matriz de coeficientes das restrições
A = [
    [1, 1, 1, 0], 
    [2, 3, 2, 1] 
]

# Vetor de termos independentes das restrições
b = [4, 5]

get_solution_simplex(c, A, b)

Iterações:  1
Problema infactivel


### Lista 13
Solução: (-1/3,10/3)

In [15]:
# Coeficientes da função objetivo
#x1+, x1-, x2, x3, x4, x5
c = [-1, 1, 1, 0, 0, 0] 

# Matriz de coeficientes das restrições
A = [
    [1, -1, 1, -1, 0, 0], 
    [-1, 1, 1, 0, 1, 0],
    [-2, 2, 1, 0, 0, -1]
]

# Vetor de termos independentes das restrições
b = [3, 4, 4]

get_solution_simplex(c, A, b)

Iterações:  3
Iterações:  1
Solução ótima:  [0, np.float64(0.33333333333333326), np.float64(3.3333333333333335), 0, np.float64(0.33333333333333326), 0]
Tempo de execução:  0.0006740093231201172


### Algum problema ilimitado


In [16]:
# Coeficientes da função objetivo
c = [3, -3, 4, -4, 0] 

# Matriz de coeficientes das restrições
A = [
    [1, -1, 2, -2, 1], 
    [2, -2, 1, -1, -1]
]

# Vetor de termos independentes das restrições
b = [8, 6]

get_solution_simplex(c, A, b)

Iterações:  2
Problema ilimitado
Solução ótima:  None
Tempo de execução:  0.0009872913360595703


# ---------------------------------------------

In [17]:
def parse_test_problem(file_path):
    """
    Parses the test.txt file and extracts the coefficients for the Simplex algorithm.

    Parameters:
        file_path (str): Path to the test.txt file.

    Returns:
        c (list): Coefficients of the objective function.
        A (list of lists): Coefficient matrix for the constraints.
        b (list): Right-hand side values for the constraints.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Initialize sections
    sections = {
        'Initial': [],
        'Aij': [],
        'Bij': [],
        'Cij': []
    }

    current_section = 'Initial'
    for line in lines:
        line = line.strip()
        if line.startswith('1st Variable Cost Component'):
            current_section = 'Aij'
            continue
        elif line.startswith('2nd Variable Cost Component'):
            current_section = 'Bij'
            continue
        elif line.startswith('Fixed Cost Component'):
            current_section = 'Cij'
            continue
        elif line == '':
            continue  # Skip empty lines
        sections[current_section].append(line)

    # Process Initial Section
    # Assuming the initial numbers represent some parameters like number of variables and constraints
    # Adjust this part based on the actual meaning of the initial numbers
    initial_params = list(map(int, sections['Initial']))
    # Example: first number could be number of variables, second number number of constraints
    # Here, we'll assume:
    # n_vars = initial_params[0]
    # n_constraints = initial_params[1]
    # Modify as per actual data structure
    n_vars = initial_params[0]
    n_constraints = initial_params[1]
    
    # Process Aij, Bij, Cij
    Aij = list(map(int, sections['Aij']))
    Bij = list(map(int, sections['Bij']))
    Cij = list(map(int, sections['Cij']))

    # Construct the coefficient matrix A and vector b
    # The exact construction depends on the problem structure
    # Here's a generic way assuming Aij and Bij are parts of constraints

    # Example construction (modify as needed):
    # Let's assume each constraint is of the form Aij * x + Bij * y <= Cij
    A = []
    b = []
    c = []

    # Assuming the number of constraints is consistent
    for i in range(n_constraints):
        A.append([Aij[i], Bij[i]])
        b.append(Cij[i])

    # Define the objective function coefficients
    # Modify this based on how the objective function is defined in test.txt
    # For example, maximize or minimize some linear combination of variables
    # Here, we'll assume a simple objective function
    c = [1 for _ in range(n_vars)]  # Example: maximize sum of variables

    return c, A, b


c, A, b = parse_test_problem("Problems/test.txt")

print(A)

[[0, 0], [6, 37]]
